In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re

In [30]:
cols = ['sentiment','id','date','query_string','user']
df = pd.read_csv('som3.csv')
# above line will be different depending on where you saved your data, and your file name
df.head()
df.drop(['retweet_count','user_followers_count','favorite_count','coordinates','geo','created_at','id_str'],axis=1,inplace=True)

In [31]:
df.columns = ['text', 'sentiment']
columnsTitles=["sentiment","text"]
df=df.reindex(columns=columnsTitles)
df[df.sentiment == 0].head(10)
df['sentiment'].fillna(0, inplace=True)
df.head()

,sentiment,text
0,0,RT @TheFuturist007: This autonomous golf ball ...
1,0,This could be Uber's 1st flying taxi 🚁\n@CNET ...
2,0,RT @bmetrock: This video will be front and cen...
3,0,Gut health testing Microba using machine learn...
4,0,RT @gp_pulipaka: Introducing Bamboolib — A GUI...


In [43]:
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes and word!='RT' and word != 'rt':
                words.append(word)
    return ' '.join(words)

def deEmojify(text):
    return text.encode('ascii', 'ignore').decode('ascii')


In [51]:
clean_df = pd.DataFrame(columns=['text','target'])
for i in range(42591): 
    value = strip_all_entities(deEmojify(strip_links(df.text[i])))
    clean_df = clean_df.append({'text': value}, ignore_index=True)
#     clean_df.text.append(strip_all_entities(deEmojify(strip_links(df.text[i]))))

In [52]:
clean_df.head()


,text,target
0,This autonomous golf ball drives itself scorin...,NaN
1,This could be Uber s 1st flying taxi via Picca...,NaN
2,This video will be front and center in tomorro...,NaN
3,Gut health testing Microba using machine learn...,NaN
4,pulipaka Introducing Bamboolib A GUI for,NaN


In [57]:
clean_df = clean_df.drop_duplicates(subset='text', keep="last").reset_index(drop=True)

In [58]:
clean_df.to_csv('clean_data.csv',encoding='utf-8')